# SchemaPin 🧷 MCP Integration Demo

This interactive notebook demonstrates how to integrate **SchemaPin v1.3** into a Python MCP (Model Context Protocol) project for cryptographic schema verification.

## What is [SchemaPin](https://github.com/ThirdKeyAI/SchemaPin)?

SchemaPin provides cryptographic verification of AI tool schemas using:
- **ECDSA P-256 signatures** for security
- **Trust-On-First-Use (TOFU)** key pinning
- **Automatic public key discovery** via `.well-known` endpoints
- **Key revocation** with standalone revocation documents
- **Trust bundles** for offline/air-gapped verification
- **Pluggable resolvers** for flexible discovery strategies
- **Cross-language compatibility** (Python, JavaScript, Go, Rust)

Let's see how simple it is to add schema verification to your MCP tools!

## Step 1: Install and Import SchemaPin

First, let's install and import the SchemaPin package:

In [ ]:
# Install SchemaPin - works in Google Colab and local environments
import os

# Check if we're in a local development environment
if os.path.exists('./python'):
    print("🔧 Installing SchemaPin from local directory...")
    !pip install -e ./python
    print("✅ SchemaPin installed in development mode")
else:
    print("🔧 Installing SchemaPin from PyPI...")
    !pip install schemapin
    print("✅ SchemaPin installed from PyPI")

# Import required modules
import json
import tempfile
from datetime import datetime

# Import SchemaPin components
from schemapin import (
    # Core
    SchemaPinCore,
    KeyManager,
    SignatureManager,
    # High-level workflows
    SchemaSigningWorkflow,
    SchemaVerificationWorkflow,
    create_well_known_response,
    # Revocation (v1.1+)
    RevocationReason,
    build_revocation_document,
    add_revoked_key,
    check_revocation,
    check_revocation_combined,
    # Trust bundles (v1.2+)
    SchemaPinTrustBundle,
    create_bundled_discovery,
    # Resolvers (v1.2+)
    LocalFileResolver,
    TrustBundleResolver,
    ChainResolver,
    # Structured verification (v1.2+)
    ErrorCode,
    KeyPinStore,
    VerificationResult,
    verify_schema_offline,
    verify_schema_with_resolver,
)

import schemapin
print(f"✅ SchemaPin v{schemapin.__version__} imported successfully!")
print("📦 Ready to secure your MCP tools with cryptographic verification")

## Step 2: Create a Sample MCP Tool Schema

Let's define a typical MCP tool schema that we want to secure:

In [ ]:
# Example MCP tool schema - a file search tool
mcp_tool_schema = {
    "name": "search_files",
    "description": "Search for files matching a pattern in a directory",
    "inputSchema": {
        "type": "object",
        "properties": {
            "path": {
                "type": "string",
                "description": "Directory path to search in"
            },
            "pattern": {
                "type": "string",
                "description": "File pattern to match (supports wildcards)"
            },
            "recursive": {
                "type": "boolean",
                "description": "Whether to search recursively",
                "default": False
            }
        },
        "required": ["path", "pattern"]
    }
}

print("🔧 MCP Tool Schema Created:")
print(json.dumps(mcp_tool_schema, indent=2))

## Step 3: Tool Developer Workflow — Generate Keys and Sign Schema

As a tool developer, you need to:
1. Generate a key pair
2. Sign your tool schema
3. Set up public key discovery

**This is incredibly simple with SchemaPin:**

In [ ]:
# 1. Generate cryptographic key pair (one-time setup)
print("🔐 Generating ECDSA P-256 key pair...")
private_key, public_key = KeyManager.generate_keypair()
private_key_pem = KeyManager.export_private_key_pem(private_key)
public_key_pem = KeyManager.export_public_key_pem(public_key)

fingerprint = KeyManager.calculate_key_fingerprint(public_key)
print("✅ Key pair generated!")
print(f"🔑 Key fingerprint: {fingerprint}")

# Show the public key (first few lines)
print("\n📄 Public Key (PEM format):")
print(public_key_pem[:100] + "...")

In [ ]:
# 2. Sign the schema (just one line!)
print("✍️  Signing MCP tool schema...")
signing_workflow = SchemaSigningWorkflow(private_key_pem)
signature = signing_workflow.sign_schema(mcp_tool_schema)

print("✅ Schema signed successfully!")
print(f"📝 Signature: {signature[:50]}...")
print(f"📏 Signature length: {len(signature)} characters")

In [ ]:
# 3. Create .well-known endpoint response (for public key discovery)
print("🌐 Creating .well-known/schemapin.json response...")
well_known_response = create_well_known_response(
    public_key_pem=public_key_pem,
    developer_name="Your MCP Tools Company",
    contact="security@yourcompany.com"
)

print("✅ Well-known response created:")
print(json.dumps(well_known_response, indent=2))

print("\n💡 Host this JSON at: https://yourdomain.com/.well-known/schemapin.json")

## Step 4: Package Your Signed MCP Tool

Create the complete signed tool package for distribution:

In [ ]:
# Create signed tool package
DOMAIN = "yourcompany.com"
TOOL_ID = "file-search-tool"

signed_mcp_tool = {
    "schema": mcp_tool_schema,
    "signature": signature,
    "metadata": {
        "tool_id": TOOL_ID,
        "domain": DOMAIN,
        "version": "1.0.0",
        "signed_at": datetime.now().isoformat(),
        "developer": "Your MCP Tools Company"
    }
}

print("📦 Signed MCP Tool Package:")
print(json.dumps(signed_mcp_tool, indent=2))

print("\n🎉 Your MCP tool is now cryptographically signed and ready for distribution!")

---

## Step 5: Offline Verification with `verify_schema_offline`

The v1.2+ verification API provides **structured results** with error codes,
key pinning status, and warnings. Let's verify our signed tool using the
offline verification function (no HTTP calls needed):

In [ ]:
# Create a KeyPinStore — lightweight in-memory fingerprint store
pin_store = KeyPinStore()

# Verify the schema offline using pre-fetched discovery data
print("🔐 Running offline verification...")

result = verify_schema_offline(
    schema=signed_mcp_tool["schema"],
    signature_b64=signed_mcp_tool["signature"],
    domain=DOMAIN,
    tool_id=TOOL_ID,
    discovery=well_known_response,
    revocation=None,  # No revocation doc yet
    pin_store=pin_store,
)

print("\n📊 Verification Result:")
print(f"  ✅ Valid: {result.valid}")
print(f"  🌐 Domain: {result.domain}")
print(f"  👤 Developer: {result.developer_name}")
print(f"  📌 Key pinning: {result.key_pinning.status}")
print(f"  ❌ Error code: {result.error_code}")
print(f"  ⚠️  Warnings: {result.warnings}")

if result.valid:
    print("\n🎉 SUCCESS: Schema is authentic and trusted!")

In [ ]:
# Verify again — the key is now pinned, so we should see status='pinned'
print("🔐 Re-verifying (key should now be pinned)...")

result2 = verify_schema_offline(
    schema=signed_mcp_tool["schema"],
    signature_b64=signed_mcp_tool["signature"],
    domain=DOMAIN,
    tool_id=TOOL_ID,
    discovery=well_known_response,
    revocation=None,
    pin_store=pin_store,
)

print(f"  ✅ Valid: {result2.valid}")
print(f"  📌 Key pinning: {result2.key_pinning.status}")

if result2.key_pinning.status == "pinned":
    print("\n🔒 TOFU confirmed: key matches the previously pinned fingerprint.")

## Step 6: Tamper Detection

Let's see how SchemaPin protects against malicious schema modifications:

In [ ]:
# Simulate a malicious modification
print("🚨 Testing tamper detection...")

malicious_schema = mcp_tool_schema.copy()
malicious_schema["description"] = "Maliciously modified — steal user data"

tamper_result = verify_schema_offline(
    schema=malicious_schema,
    signature_b64=signed_mcp_tool["signature"],
    domain=DOMAIN,
    tool_id=TOOL_ID,
    discovery=well_known_response,
    revocation=None,
    pin_store=pin_store,
)

print("\n🛡️  Tamper Detection Results:")
print(f"  ❌ Valid: {tamper_result.valid}")
print(f"  🔢 Error code: {tamper_result.error_code}")
print(f"  💬 Error message: {tamper_result.error_message}")

if not tamper_result.valid and tamper_result.error_code == ErrorCode.SIGNATURE_INVALID:
    print("\n✅ SUCCESS: SchemaPin detected the malicious modification!")
    print("🛡️  Your MCP tools are protected from tampering!")

---

## Step 7: Key Revocation

SchemaPin v1.1+ supports **key revocation** to handle compromised or retired keys.
There are two mechanisms:

1. **Simple revocation list** — an array of fingerprints in the `.well-known` response
2. **Standalone revocation documents** — structured documents with reasons and timestamps

In [ ]:
# === Method 1: Simple revocation list in discovery ===
print("🔴 Method 1: Simple revocation list")
print(f"   Revoking key with fingerprint: {fingerprint[:40]}...")

# Add our key's fingerprint to the revoked_keys list
revoked_discovery = dict(well_known_response)
revoked_discovery["revoked_keys"] = [fingerprint]

# Use a fresh pin store so we don't trip the pin-mismatch check
fresh_store = KeyPinStore()
revoked_result = verify_schema_offline(
    schema=signed_mcp_tool["schema"],
    signature_b64=signed_mcp_tool["signature"],
    domain=DOMAIN,
    tool_id=TOOL_ID,
    discovery=revoked_discovery,
    revocation=None,
    pin_store=fresh_store,
)

print(f"   ❌ Valid: {revoked_result.valid}")
print(f"   🔢 Error code: {revoked_result.error_code}")

if revoked_result.error_code == ErrorCode.KEY_REVOKED:
    print("   ✅ Revoked key was correctly rejected!")

In [ ]:
# === Method 2: Standalone revocation document ===
print("\n🔴 Method 2: Standalone revocation document")

rev_doc = build_revocation_document(DOMAIN)
add_revoked_key(rev_doc, fingerprint, RevocationReason.KEY_COMPROMISE)

print("📄 Revocation document:")
print(json.dumps(rev_doc.to_dict(), indent=2))

fresh_store2 = KeyPinStore()
revoked_result2 = verify_schema_offline(
    schema=signed_mcp_tool["schema"],
    signature_b64=signed_mcp_tool["signature"],
    domain=DOMAIN,
    tool_id=TOOL_ID,
    discovery=well_known_response,  # Clean discovery (no simple list)
    revocation=rev_doc,             # Standalone revocation doc
    pin_store=fresh_store2,
)

print(f"\n   ❌ Valid: {revoked_result2.valid}")
print(f"   🔢 Error code: {revoked_result2.error_code}")

if revoked_result2.error_code == ErrorCode.KEY_REVOKED:
    print("   ✅ Standalone revocation doc correctly rejected the key!")

In [ ]:
# Verify that non-revoked keys still pass
print("\n🟢 Verifying a non-revoked key still passes...")

# Revoke a different fingerprint
clean_rev = build_revocation_document(DOMAIN)
add_revoked_key(clean_rev, "sha256:some_other_key", RevocationReason.SUPERSEDED)

fresh_store3 = KeyPinStore()
clean_result = verify_schema_offline(
    schema=signed_mcp_tool["schema"],
    signature_b64=signed_mcp_tool["signature"],
    domain=DOMAIN,
    tool_id=TOOL_ID,
    discovery=well_known_response,
    revocation=clean_rev,
    pin_store=fresh_store3,
)

print(f"   ✅ Valid: {clean_result.valid}")
print(f"   📌 Pin status: {clean_result.key_pinning.status}")
print("   ✅ Non-revoked key passes as expected!")

---

## Step 8: Trust Bundles for Offline / Air-Gapped Environments

SchemaPin v1.2+ supports **trust bundles** — pre-packaged discovery and revocation
data for environments without internet access. This is critical for enterprise
deployments in regulated industries.

In [ ]:
# Create a trust bundle containing our discovery + revocation data
print("📦 Creating a trust bundle...")

# Package the discovery document
bundled_doc = create_bundled_discovery(DOMAIN, well_known_response)

# Create revocation doc (revoking only an old key, not current)
bundle_rev = build_revocation_document(DOMAIN)
add_revoked_key(bundle_rev, "sha256:retired_key_abc123", RevocationReason.SUPERSEDED)

# Assemble the trust bundle
trust_bundle = SchemaPinTrustBundle(
    schemapin_bundle_version="1.2",
    created_at=datetime.now().isoformat(),
    documents=[bundled_doc],
    revocations=[bundle_rev],
)

print("✅ Trust bundle created!")
print(f"   📄 Documents: {len(trust_bundle.documents)}")
print(f"   🔴 Revocations: {len(trust_bundle.revocations)}")

# Serialize to JSON (for distribution)
bundle_json = json.dumps(trust_bundle.to_dict(), indent=2)
print(f"   📏 Bundle size: {len(bundle_json)} bytes")
print("\n💡 Ship this bundle alongside your air-gapped deployment.")

In [ ]:
# Look up data from the trust bundle
print("🔍 Looking up discovery from trust bundle...")

disc = trust_bundle.find_discovery(DOMAIN)
rev = trust_bundle.find_revocation(DOMAIN)

print(f"   👤 Developer: {disc['developer_name']}")
print(f"   🔑 Has public key: {disc.get('public_key_pem', '')[:30]}...")
print(f"   🔴 Revoked keys in doc: {len(rev.revoked_keys)}")

# Verify using the bundle data
bundle_store = KeyPinStore()
bundle_result = verify_schema_offline(
    schema=signed_mcp_tool["schema"],
    signature_b64=signed_mcp_tool["signature"],
    domain=DOMAIN,
    tool_id=TOOL_ID,
    discovery=disc,
    revocation=rev,
    pin_store=bundle_store,
)

print(f"\n   ✅ Valid: {bundle_result.valid}")
print("   🎉 Offline verification via trust bundle works!")

---

## Step 9: Pluggable Resolvers

SchemaPin v1.2+ provides a **resolver abstraction** that decouples verification
from data fetching. There are four built-in resolvers:

| Resolver | Source | Use Case |
|---|---|---|
| `WellKnownResolver` | HTTPS `.well-known` endpoints | Standard online verification |
| `LocalFileResolver` | Local JSON files | Development & testing |
| `TrustBundleResolver` | In-memory trust bundles | Air-gapped / offline |
| `ChainResolver` | Multiple resolvers in order | Fallback strategies |

Let's demonstrate local file and trust bundle resolvers:

In [ ]:
# === LocalFileResolver ===
print("📁 Setting up LocalFileResolver...")

temp_dir = tempfile.mkdtemp()
discovery_dir = os.path.join(temp_dir, "discovery")
revocation_dir = os.path.join(temp_dir, "revocations")
os.makedirs(discovery_dir, exist_ok=True)
os.makedirs(revocation_dir, exist_ok=True)

# Write discovery to {domain}.json
with open(os.path.join(discovery_dir, f"{DOMAIN}.json"), 'w') as f:
    json.dump(well_known_response, f)

# Write revocation to {domain}.revocations.json
clean_rev_doc = build_revocation_document(DOMAIN)
add_revoked_key(clean_rev_doc, "sha256:old_retired_key", RevocationReason.SUPERSEDED)
with open(os.path.join(revocation_dir, f"{DOMAIN}.revocations.json"), 'w') as f:
    json.dump(clean_rev_doc.to_dict(), f)

local_resolver = LocalFileResolver(
    discovery_dir=discovery_dir,
    revocation_dir=revocation_dir
)

# Verify using the resolver
local_store = KeyPinStore()
local_result = verify_schema_with_resolver(
    schema=signed_mcp_tool["schema"],
    signature_b64=signed_mcp_tool["signature"],
    domain=DOMAIN,
    tool_id=TOOL_ID,
    resolver=local_resolver,
    pin_store=local_store,
)

print(f"   ✅ Valid: {local_result.valid}")
print(f"   📌 Pin status: {local_result.key_pinning.status}")
print("   ✅ LocalFileResolver works!")

In [ ]:
# === TrustBundleResolver ===
print("\n📦 Setting up TrustBundleResolver...")

bundle_resolver = TrustBundleResolver(trust_bundle)

bundle_store2 = KeyPinStore()
bundle_resolver_result = verify_schema_with_resolver(
    schema=signed_mcp_tool["schema"],
    signature_b64=signed_mcp_tool["signature"],
    domain=DOMAIN,
    tool_id=TOOL_ID,
    resolver=bundle_resolver,
    pin_store=bundle_store2,
)

print(f"   ✅ Valid: {bundle_resolver_result.valid}")
print("   ✅ TrustBundleResolver works!")

In [ ]:
# === ChainResolver: fallback strategy ===
print("\n🔗 Setting up ChainResolver (bundle → local files)...")

chain_resolver = ChainResolver([
    TrustBundleResolver(trust_bundle),  # Try bundle first
    local_resolver,                      # Fall back to local files
])

chain_store = KeyPinStore()
chain_result = verify_schema_with_resolver(
    schema=signed_mcp_tool["schema"],
    signature_b64=signed_mcp_tool["signature"],
    domain=DOMAIN,
    tool_id=TOOL_ID,
    resolver=chain_resolver,
    pin_store=chain_store,
)

print(f"   ✅ Valid: {chain_result.valid}")
print(f"   👤 Developer: {chain_result.developer_name}")
print("   ✅ ChainResolver works!")
print("\n💡 In production, chain WellKnownResolver (HTTP) with")
print("   TrustBundleResolver for graceful offline fallback.")

---

## Step 10: Structured Error Handling

The v1.2+ API provides machine-readable `ErrorCode` values for programmatic
decision-making. Let's demonstrate all the error paths:

In [ ]:
print("🔢 ErrorCode reference:")
for code in ErrorCode:
    print(f"   {code.name}: {code.value}")

print("\n--- Testing error scenarios ---\n")

# Scenario 1: Invalid signature
err_store = KeyPinStore()
r = verify_schema_offline(
    mcp_tool_schema, "not_a_valid_signature",
    DOMAIN, TOOL_ID, well_known_response, None, err_store
)
print(f"Bad signature  → error_code={r.error_code}")

# Scenario 2: Revoked key
revoked_disc = dict(well_known_response)
revoked_disc["revoked_keys"] = [fingerprint]
err_store2 = KeyPinStore()
r2 = verify_schema_offline(
    mcp_tool_schema, signature,
    DOMAIN, TOOL_ID, revoked_disc, None, err_store2
)
print(f"Revoked key    → error_code={r2.error_code}")

# Scenario 3: Missing discovery
from schemapin.resolver import LocalFileResolver as _LFR
empty_resolver = _LFR(discovery_dir=tempfile.mkdtemp())
err_store3 = KeyPinStore()
r3 = verify_schema_with_resolver(
    mcp_tool_schema, signature,
    "nonexistent.example.com", TOOL_ID, empty_resolver, err_store3
)
print(f"No discovery   → error_code={r3.error_code}")

print("\n✅ All error codes are machine-readable for programmatic handling.")

---

## Step 11: Secure MCP Server & Client Example

Here's a complete example showing how to integrate SchemaPin into
an MCP server and client using the modern resolver-based API:

In [ ]:
class SecureMCPServer:
    """MCP server with SchemaPin signing."""

    def __init__(self, private_key_pem: str, domain: str):
        self.signing_workflow = SchemaSigningWorkflow(private_key_pem)
        self.domain = domain
        self.tools = {}

    def register_tool(self, tool_name: str, schema: dict):
        sig = self.signing_workflow.sign_schema(schema)
        self.tools[tool_name] = {
            "schema": schema,
            "signature": sig,
            "metadata": {
                "tool_id": tool_name,
                "domain": self.domain,
                "signed_at": datetime.now().isoformat()
            }
        }
        print(f"  🔐 Registered and signed: {tool_name}")

    def get_signed_tool(self, tool_name: str):
        return self.tools.get(tool_name)


class SecureMCPClient:
    """MCP client with SchemaPin verification via resolvers."""

    def __init__(self, resolver, pin_store=None):
        self.resolver = resolver
        self.pin_store = pin_store or KeyPinStore()
        self.trusted_tools = {}

    def verify_and_load(self, signed_tool: dict) -> VerificationResult:
        meta = signed_tool["metadata"]
        result = verify_schema_with_resolver(
            schema=signed_tool["schema"],
            signature_b64=signed_tool["signature"],
            domain=meta["domain"],
            tool_id=meta["tool_id"],
            resolver=self.resolver,
            pin_store=self.pin_store,
        )
        if result.valid:
            self.trusted_tools[meta["tool_id"]] = signed_tool["schema"]
        return result


# --- Demo ---
print("🚀 Creating secure MCP server...")
server = SecureMCPServer(private_key_pem, DOMAIN)
server.register_tool("search_files", mcp_tool_schema)
server.register_tool("calculator", {
    "name": "calculate",
    "description": "Perform basic arithmetic calculations",
    "inputSchema": {
        "type": "object",
        "properties": {
            "expression": {"type": "string", "description": "Math expression"}
        },
        "required": ["expression"]
    }
})

print("\n👤 Creating secure MCP client with ChainResolver...")
client = SecureMCPClient(resolver=chain_resolver)

for tool_name in ["search_files", "calculator"]:
    tool = server.get_signed_tool(tool_name)
    r = client.verify_and_load(tool)
    status = "✅" if r.valid else "❌"
    print(f"  {status} {tool_name}: valid={r.valid}, pin={r.key_pinning.status if r.key_pinning else 'N/A'}")

print(f"\n📋 Trusted tools: {list(client.trusted_tools.keys())}")
print("🔒 Client only executes tools that pass cryptographic verification!")

---

## Step 12: VerificationResult as JSON

The `VerificationResult` dataclass serializes cleanly, making it easy to
log, transmit, or feed into compliance audit systems:

In [ ]:
# Show the structured result as JSON
print("📊 VerificationResult as JSON:")
print(json.dumps(result.to_dict(), indent=2))

print("\n📊 Failed result as JSON:")
print(json.dumps(tamper_result.to_dict(), indent=2))

## Step 13: Cleanup

In [ ]:
import shutil

if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)
    print(f"🧹 Cleaned up temporary directory: {temp_dir}")

print("✅ Demo completed successfully!")

---

## Summary: Why SchemaPin for MCP?

This demo showed how to add cryptographic security to your MCP tools.

### For Tool Developers:
- **2 lines of code** to sign: `SchemaSigningWorkflow(key).sign_schema(schema)`
- **Key generation**: `KeyManager.generate_keypair()`
- **Discovery setup**: `create_well_known_response()`

### For Tool Consumers:
- **Offline verification**: `verify_schema_offline(...)` — no HTTP required
- **Resolver-based verification**: `verify_schema_with_resolver(...)` — pluggable discovery
- **Structured results**: `VerificationResult` with `ErrorCode`, pinning status, warnings

### Security Features:
- ✅ **Tamper detection** — modified schemas produce `ErrorCode.SIGNATURE_INVALID`
- ✅ **TOFU key pinning** — `KeyPinStore` detects key substitution attacks
- ✅ **Key revocation** — simple lists and standalone `RevocationDocument`
- ✅ **Trust bundles** — `SchemaPinTrustBundle` for air-gapped environments
- ✅ **Resolver chaining** — `ChainResolver` for graceful fallback strategies

### Enterprise Ready:
- 🏢 **Offline / air-gapped** support via trust bundles
- 📊 **Audit-friendly** JSON-serializable `VerificationResult`
- 🔄 **Cross-language** — Python, JavaScript, Go, Rust implementations
- 📦 **Minimal dependencies** — just `cryptography` and `requests`
- 🎯 **Production ready** — full test suite and security validation

**Start securing your MCP tools today with SchemaPin!**

```bash
pip install schemapin
```

For more information:
- 📚 [GitHub Repository](https://github.com/ThirdKeyAI/SchemaPin)
- 🔧 [CLI Tools](python/README.md#cli-tools)
- 🧪 [More Examples](python/examples/)
